# torchquad - Example Notebook

torchquad is a dedicated module for numerical integration in arbitrary dimensions.

This notebook gives more detailed look at its functionality and explores some performance considerations as well.

The main problem with higher-dimensional numerical integration is that the computation simply becomes too costly if *n* is large as the number of evaluation points increases exponentially &#151; this problem is known as the *curse of dimensionality*. This especially affects grid-based methods, but is - to some degree- also present for Monte Carlo methods, which also require larger numbers of points for convergence in higher dimensions. 

At this time, torchquad offers the following integration methods for abritrary dimensionality.

| Common name        | How it works                                                      | Spacing |
|--------------------|-------------------------------------------------------------------|---------|
| Trapezoid rule     | Creates a linear interpolant between two neighbouring points      | Equal   |
| Simpson's rule     | Creates a quadratic interpolant between three neighbouring points | Equal   |
| Monte Carlo        | Randomly chooses points at which the integrand is evaluated       | Random  |

### Outline 
This notebook is a guide for new users to torchquad and is structured in the following way:

* Example integration using Monte Carlo integration in one dimension (1-D)  
* Example integration in five dimensions (10-D)  
* Some accuracy / runtime comparisons with `scipy`

## Imports
Now let's get started! First, the general imports:

In [ ]:
import scipy

#For plotting
import matplotlib.pyplot as plt

# To avoid copying things to GPU memory, 
# ideally allocate everything in torch on the GPU
# and avoid non torch function calls
import torch
torch.set_printoptions(precision=10) # Set displayed output precision to 10 digits

from torchquad import enable_cuda #necessary to enable GPU support
from torchquad import MonteCarlo, Trapezoid, Simpson # The integrators we will use
import torchquad

In [ ]:
enable_cuda() #Use this to enable GPU support. 

## One dimensional integration

For clarity on the methods used in this notebook, we will start with two examples in one dimension. See [Patrick Walls][1]' nice Python introduction to the [Trapezoid rule][2] and [Simpson's rule][3] in one dimension. 

Let `f` be the function $ f(x) = e^x \cdot x^2 $
Over the domain $[0,2]$, the integral of `f` is $\int_{0}^{2} e^x \cdot x^2 \,dx = 2(e^2-1) = 12.7781121978613004544... $.  

[1]: https://github.com/patrickwalls
[2]: https://www.math.ubc.ca/~pwalls/math-python/integration/trapezoid-rule/
[3]: https://www.math.ubc.ca/~pwalls/math-python/integration/simpsons-rule/

Let's declare the function, remember the correct result and a simple function to print the absolute error.

In [ ]:
def f(x):
    return torch.exp(x) * torch.pow(x,2)

def print_error(result,solution):
    print("Results:",result)
    print("Abs. Error:",torch.abs(result - solution))

solution = 2*(torch.exp(torch.tensor([2.]))-1)

**Note that we are using the torch versions to ensure that all variables are and stay on the GPU.**

Let's plot the function briefly.

In [ ]:
points = torch.linspace(0,2,100)
plt.plot(points.cpu(),f(points).cpu()) #Note that for plotting we have to move the values to the CPU first

Let's define the integration domain now and initialize the integrator.

In [ ]:
integration_domain = [[0, 2]] #Integration domain always is a list of lists to allow arbitrary dimensionality.
tp = Trapezoid()  #Initialize a trapezoid solver

Now we are all set to compute the integral. Let's try it with just 11 sample points for now.

In [ ]:
result = tp.integrate(f, dim=1, N=101, integration_domain=integration_domain)
print_error(result,solution)

Alright, quite close already as 1-D integrals are comparatively easy. Let's see what type of value we get for different integrators.

In [ ]:
simp = Simpson()
result = simp.integrate(f, dim=1, N=101, integration_domain=integration_domain)
print_error(result,solution)

In [ ]:
mc = MonteCarlo()
result = mc.integrate(f, dim=1, N=101, integration_domain=integration_domain)
print_error(result,solution)

Notably, Simpson's method is already sufficient for a perfect solutiuon here with 101 points. Monte Carlo does not perform so well, it is more suited to higher dimensional integrals as we will see.

Note that we picked 101 points as the implemented Simpson's method currently only supports odd numbers of points. Let's step things up now and move to a 10-dimensional problem.

## High-dimensional integration

For this we will investigate the following: 

Let `f_2` be the function $ f_2(x) = \sum_{i=1}^{10} sin(x_i) $

Over the domain $[0,1]^{10}$, the integral of `f_2` is $\int_{0}^{1}...\int_{0}^{1} \sum_{i=1}^{10} sin(x_i) = 20sin^2(1/2) = 4.59697694131860282599063392557... $.  

Plotting this is tricky, so let's directly move to the integrals.

In [ ]:
def f_2(x):
    return torch.sum(torch.sin(x),dim=1)

solution = 20*(torch.sin(torch.tensor([0.5]))*torch.sin(torch.tensor([0.5])))

Let's start with just 5 points per dimension, i.e. $5^10=9,765,625$ points. You can see the curse of dimensionality fully at play here. 

*N.B. Currently, torchquad only support equal numbers of points per dimension. We are on it.*

In [ ]:
integration_domain = [[0, 1]]*10 #Integration domain always is a list of lists to allow arbitrary dimensionality.
N = 5**10 

In [ ]:
tp = Trapezoid()  #Initialize a trapezoid solver
result = tp.integrate(f_2, dim=10, N=N, integration_domain=integration_domain)
print_error(result,solution)

In [ ]:
simp = Simpson()  #Initialize a trapezoid solver
result = simp.integrate(f_2, dim=10, N=N, integration_domain=integration_domain)
print_error(result,solution)

In [ ]:
mc = MonteCarlo()
result = mc.integrate(f_2, dim=10, N=N, integration_domain=integration_domain)
print_error(result,solution)

Note that the simple Monte Carlo method is much more competitive for this case. The bad convergence properties of the trapezoid method are visible while simpson is still ok given the comparatively smooth integrand.

## Comparison with scipy

Let's explore, how torchquad's performance compares to scipy the go-to tool for numerical integration. A more detailed exploration of this topic can be found in the *Evaluation_notebook.ipynb*.

TODO